# Présentation

Le présent notebook traite le coté prétraitement; on parlera de la dataset, le processus de chargement, le filtrage (stop words), ainsi que les opérations de tokénisation, racinalisation (steeming) et Lematization.

En fin, on exportera le résultat dans un fichier csv qui sera par la suite utilisé pour effectuer le reste des opérations demandées (exploration, visualisation et analyse).

# Initialisation

##### $Les$ $imports$ $necessaires$

In [56]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

### $Le$ $dataset$

Le dataset (disponible içi [1]) contient plus de 13k commentaires sur Disneyland Paris.

Les colonnes sont : 
- Rating (note sur 5), 
- Year_Month (mois du commentaire), 
- Reviewer_Location (Pays d'origine du commentaire), 
- Review_Text (contenu du commentaire)

In [57]:
# On récupère le fichier
url = 'https://assets-datascientest.s3-eu-west-1.amazonaws.com/de/total/reviews.csv'
r = requests.get(url, allow_redirects=True)

open('reviews.csv', 'wb').write(r.content)

12732861

### $Extraction$ $des$ $données$ $et$ $apperçu$

In [58]:
df = pd.read_csv('reviews.csv')
print('Dataset Shape :', df.shape)
print('Dataset overview :')
display (df.head())

Dataset Shape : (13630, 4)
Dataset overview :


,Rating,Year_Month,Reviewer_Location,Review_Text
0,5,2019-3,United Arab Emirates,"We've been to Disneyland Hongkong and Tokyo, s..."
1,4,2018-6,United Kingdom,I went to Disneyland Paris in April 2018 on Ea...
2,5,2019-4,United Kingdom,"What a fantastic place, the queues were decent..."
3,4,2019-4,Australia,We didn't realise it was school holidays when ...
4,5,missing,France,A Trip to Disney makes you all warm and fuzzy ...


## La propriété Target (Sentiment)

In [59]:
df['sentiment'] = df['Rating'] > 2.5
df.sentiment = df.sentiment.map({True:1, False:0})
print(df.sentiment.value_counts(normalize=True), '\n')

1    0.862656
0    0.137344
Name: sentiment, dtype: float64 



## Modification des labels (par commodité)

In [60]:
# Let us add a new column
df['text'] = df.Review_Text
txt = df.text[0]
print(txt)

We've been to Disneyland Hongkong and Tokyo, so far this one is the best. We're looking forward to visit the biggest one in Orlando. 1 day is not enough, it is recommended to stay in Disney Hotel   Resort so you can enjoy the fast track.. save huge amount of time.. if you're not staying there, plan and strategize your visit by getting all the fast track passes from kiosk nearby the attraction; then come back when it's time for your ride. The projection and fireworks show are out of this world!!


# Text Normalization 

### $Initialisation$ $et$ $packages$

La bibliothèque nltk

Natural Language Tool Kit (NLTK) is a Python library to make programs that work with natural language. 
It provides a user-friendly interface to datasets that are over 50 corpora and lexical resources 
such as WordNet Word repository. The library can perform different operations such as tokenizing, 
stemming, classification, parsing, tagging, and semantic reasoning [2].

Natural Language Tool Kit (NLTK) est une bibliothèque Python permettant de créer des programmes qui fonctionnent avec le langage naturel. Elle fournit une interface à des datasets avec plus de 50 corpus et ressources lexicales comme, e.g. WordNet. La bibliothèque peut effectuer différentes opérations telles que la tokénisation, la déformation, la classification, l'analyse syntaxique, le balisage et le raisonnement sémantique, etc [2].

*Instalation :* 

`sudo pip install -U nltk to install nltk on Mac or Linux`

`or run pip install nltk on your cmd.exe bash on Windows.`

In [61]:
import nltk

from nltk import FreqDist

from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Le Filtrage Stop Words
# Importer stopwords de la classe nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stemming (plus de détails ci-dessous)
from nltk.stem import LancasterStemmer

# Lematisation
nltk.download('wordnet')
from nltk import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     /home/abdelbasset/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abdelbasset/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/abdelbasset/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### $La$ $Tokenisation$

La tokénisation consiste par définition,  à découper en morceaux appelés tokens une séquence de caractères en éliminant éventuellement certains caractères comme la ponctuation [3]. 

In [62]:
def simple_tokenize(txt):
    mots = word_tokenize(txt)
    return mots

def regex_tokenize(txt):
    tokenizer = RegexpTokenizer(r'\w+')
    mots = tokenizer.tokenize(txt)    
    return mots

def regex_delete_numeric(txt):
    tokenizer = RegexpTokenizer(r'[^\d]+')
    mots = tokenizer.tokenize(txt)    
    return mots

start = time.time()
mega_mots = list(map(regex_tokenize, df.text))
end = time.time()
print('Calculation done in :', round(end - start, 2), 's') # ==> 1.37s 1.24 1.33 1.24 1.25
print('First line => Size = ', len(mega_mots[0]))
print(mega_mots[0])
len(mega_mots) # ==> 13630

Calculation done in : 1.25 s
First line => Size =  95
['We', 've', 'been', 'to', 'Disneyland', 'Hongkong', 'and', 'Tokyo', 'so', 'far', 'this', 'one', 'is', 'the', 'best', 'We', 're', 'looking', 'forward', 'to', 'visit', 'the', 'biggest', 'one', 'in', 'Orlando', '1', 'day', 'is', 'not', 'enough', 'it', 'is', 'recommended', 'to', 'stay', 'in', 'Disney', 'Hotel', 'Resort', 'so', 'you', 'can', 'enjoy', 'the', 'fast', 'track', 'save', 'huge', 'amount', 'of', 'time', 'if', 'you', 're', 'not', 'staying', 'there', 'plan', 'and', 'strategize', 'your', 'visit', 'by', 'getting', 'all', 'the', 'fast', 'track', 'passes', 'from', 'kiosk', 'nearby', 'the', 'attraction', 'then', 'come', 'back', 'when', 'it', 's', 'time', 'for', 'your', 'ride', 'The', 'projection', 'and', 'fireworks', 'show', 'are', 'out', 'of', 'this', 'world']


13630

### $Le$ $Filtrage$ $Stop$ $Words$

In [63]:
# Initialiser la variable des mots vides
stop_words = set(stopwords.words('english'))

# Ajouter plus de mots pour ne garder que des mots significatives dans notre context
extention = {'pass', 'also', 'see', 'take', 'get', 'paris', 'park', 'find', 'still', 'even', 'staff', \
             'u', 'year', 'although', 'experience', 'use', 'really', 'place', 'hotel', 'visit', 'etc', 'thing', \
             'would', 'could', 'though', 'disneyland', 'make', 'however', 'people', 'around', 'us', 'look',
             'know', 'line', 'think', 'theme', 'things', 'character', 'say', 'disney', 'ticket', 'many', 'lot', \
             'one', 'euro', 'restaurant', 'everything', 'always', 'hour', 'first', 'studio', 'tell', 'time',\
             'another', 'back', 'give', 'adult', 'children', 'daughter', 'work', 'two', 'go', 'open', 'village',\
             'come', 'mean', 'meet', 'try', 'especially', 'food', 'rid', 'kid', 'rid', 'way', 'start', 'spend', \
             'book', 'seem', 'plan', 'hours', 'studios', 'day', 'manage', 'minute', 'family', 'trip', 'return',\
             'whole', 'arrive', 'florida', 'different', 'area', 'restaurants', 'last', 'child',\
             'attraction', 'queue', 'days', 'eat', 'world'}

extended_sw = stop_words.copy()
extended_sw.update(extention)
#print(sorted(c))

def stop_words_filetering(list, stop_words):
    cleaned_list = [word.lower() for word in list if word.lower() not in stop_words]
    return cleaned_list

def delete_numeric(list):
    cleaned_list = [word.lower() for word in list if not word.lower().isdigit()]
    return cleaned_list


start = time.time()
mega_mots = list(map(lambda m: stop_words_filetering(m, extended_sw), mega_mots))                
mega_words = list(map(delete_numeric, mega_mots))                 
end = time.time()

# ----------------
# Check output
print('Results after :', round(end - start, 2), 's') # ==> 1.51s/1.04s/1.25/1.86/1.26/1.29/1.39
print('First line => Size = ', len(mega_words[0]))
print(mega_words[0])
len(mega_words) # ==> 13630
# --------------------------------

Results after : 1.39 s
First line => Size =  30
['hongkong', 'tokyo', 'far', 'best', 'looking', 'forward', 'biggest', 'orlando', 'enough', 'recommended', 'stay', 'resort', 'enjoy', 'fast', 'track', 'save', 'huge', 'amount', 'staying', 'strategize', 'getting', 'fast', 'track', 'passes', 'kiosk', 'nearby', 'ride', 'projection', 'fireworks', 'show']


13630

### $La$ $Racinisation$ $(Stemming)$

La racinisation est en fait, le procédé de transformation des flexions en leur radical ou racine. La racine
d’un mot correspond à la partie du mot restante une fois que l’on a supprimé son (ses) préfixe(s) et suffixe(s), 
à savoir son radical [4].

*`Stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root 
form—generally a written word form [3]`*

Il existe des Stemmers anglais et non-anglais disponibles dans le paquetage nltk [6]. Pour l'anglais, nous pouvons choisir entre **PorterStammer** et **LancasterStammer**, PorterStemmer étant le plus ancien, développé à l'origine en 1979. LancasterStemmer a été développé en 1990 et utilise une approche plus agressive que l'algorithme de Porter. Dans ce qui suit, nous optons pour **LancasterStammer** [6].

**$n.b.$** On peut générer son propre ensemble de règles pour n'importe quelle langue, c'est pourquoi Python nltk a introduit les **Snowball Stemmers** qui sont utilisés pour créer des stemmers non anglais [6].

In [64]:
def steem_it(words) :
    #stemmer = EnglishStemmer()
    stemmer = LancasterStemmer()
    results = [stemmer.stem(word) for word in words]
    return results
    
start = time.time()
mega_steemed_words = list(map(steem_it, mega_words))                 
end = time.time()

# ----------------
# Check output
print('Calculation done in :', round(end - start, 2), 's') # ==> 21s/19.11s/33.9s/31.47s/33.13s
print('First line => Size = ', len(mega_steemed_words[0]))
print(mega_steemed_words[0])
len(mega_steemed_words) # ==> 13630
# --------------------------------

Calculation done in : 33.13 s
First line => Size =  30
['hongkong', 'tokyo', 'far', 'best', 'look', 'forward', 'biggest', 'orlando', 'enough', 'recommend', 'stay', 'resort', 'enjoy', 'fast', 'track', 'sav', 'hug', 'amount', 'stay', 'strategize', 'get', 'fast', 'track', 'pass', 'kiosk', 'nearby', 'rid', 'project', 'firework', 'show']


13630

### $La$ $Lemmatisation$

La lemmatisation désigne un traitement lexical apporté à un texte en vue de son analyse. Ce traitement consiste à appliquer aux occurrences des lexèmes sujets à flexion (en français, verbes, substantifs, adjectifs) un codage renvoyant à leur entrée lexicale commune (« forme canonique » enregistrée dans les dictionnaires de la langue, le plus couramment), que l'on désigne sous le terme de lemme [7].

*`Lemmatisation (or lemmatization) in linguistics is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form [6].`*

NLTK fournis **WordNet** un lemmatizer qui permet d'utiliser la base de données WordNet pour rechercher des lemmes de mots.

Attention au téléchargement de wordnet, nécéssaire avant d'utiliser le Lemmatizer, i.e.

`nltk.download('wordnet')`

In [65]:
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize_it(words) :
    results = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]
    return results
    
start = time.time()
mega_lemmatized_words = list(map(lemmatize_it, mega_words))
end = time.time()

# ----------------
# Check output
print('Calculation done in :', round(end - start, 2), 's') # ==> 11s/9.99/11.6s/9./6.58s/10.33s
print('First line => Size = ', len(mega_lemmatized_words[0]))
print(mega_lemmatized_words[0])
len(mega_lemmatized_words) # ==> 13630
# --------------------------------


Calculation done in : 10.33 s
First line => Size =  30
['hongkong', 'tokyo', 'far', 'best', 'look', 'forward', 'biggest', 'orlando', 'enough', 'recommend', 'stay', 'resort', 'enjoy', 'fast', 'track', 'save', 'huge', 'amount', 'stay', 'strategize', 'get', 'fast', 'track', 'pass', 'kiosk', 'nearby', 'ride', 'projection', 'fireworks', 'show']


13630

#### $Stemming$ $vs$ $Lematization$

Contrairement au $lemme$ qui correspond à un terme issu de l’usage ordinaire des locuteurs de la langue, la racine $(stem)$ ne correspond généralement qu’à un terme résultant de ce type d’analyse. Par exemple, le mot $chercher$ a pour radical $cherch$ qui ne correspond pas à un terme employé en dehors d’une référence à ce radical même. Dans des cas particuliers, le radical peut coïncider avec un terme de vocabulaire ordinaire. C’est par exemple le cas de $frontal$ qui donne la racine $front$ [9]. 

Du coté choix, l'utilisation du stemming ou du la lemmatisation dépend fortement de nos besoins spécifiques [10]. En général, les avantages du stemming sont qu'il est simple à mettre en œuvre et rapide à exécuter. La contrepartie est que le résultat peut contenir des inexactitudes, même si elles ne sont pas pertinentes pour certaines tâches, e.g indexation de textes.

La lemmatisation par contre, fournit de meilleurs résultats en effectuant une analyse qui produit de vrais mots de dictionnaire. L'opération est par conséquent plus difficile à mettre en œuvre et plus lente que le stemming. 

En résumé, la lemmatisation est presque toujours un meilleur choix d'un point de vue qualitatif. Avec les ressources informatiques actuelles, l'exécution d'algorithmes de lemmatisation ne devrait pas avoir d'impact significatif sur les performances globales. Cependant, si nous optimisons fortement la vitesse, un algorithme de lemmatisation plus simple peut être une possibilité [10].

### $Reconversion$ $vers$ $du$ $text$ $à$ $nouveau$

Après la normalisation, on revient vers la forme textuelle brute avant d\'exporter notre dataframe vers un csv standard

In [66]:
def list_2_str(mylist):
    # Définir la variable text
    text = ""
    for word in mylist : 
        text += word+ ' '
    return text

df.text = list(map(lambda alist : ' '.join(alist), mega_lemmatized_words))
#mega_mots = list(map(list_2_str, mega_lemmatized_words))
#mega_mots[0]
#df.text = mega_mots 

### $Sauvgarde$ $et$ $exportation$ $vers$ $du$ $csv$

In [68]:
#df[['text', 'sentiment']].head()
df.drop(['Review_Text'], axis=1, inplace=True)
# Sauvegarde du dataset
df.to_csv('encoded_reviews.csv', index=False)
df.head()

,Rating,Year_Month,Reviewer_Location,sentiment,text
0,5,2019-3,United Arab Emirates,1,hongkong tokyo far best look forward biggest o...
1,4,2018-6,United Kingdom,1,go april easter weekend say june choose date l...
2,5,2019-4,United Kingdom,1,fantastic queue decent best apparently manage ...
3,4,2019-4,Australia,1,realise school holiday go consequently extreme...
4,5,missing,France,1,make warm fuzzy actual big make fun fill happy...


# References

- **[1]  :** https://assets-datascientest.s3-eu-west-1.amazonaws.com/de/total/reviews.csv
- **[2]  :** https://www.datacamp.com/community/tutorials/stemming-lemmatization-python
- **[3]  :** https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html

- **[4]  :** https://fr.wikipedia.org/wiki/Racinisation
- **[5]  :** https://en.wikipedia.org/wiki/Stemming
- **[6]  :** https://www.datacamp.com/community/tutorials/stemming-lemmatization-python
- **[7]  :** https://fr.wikipedia.org/wiki/Lemmatisation
- **[8]  :** https://en.wikipedia.org/wiki/Lemmatisation

- **[9]  :** https://fr.wikipedia.org/wiki/Lemmatisation 

- **[10] :** https://www.baeldung.com/cs/stemming-vs-lemmatization